In [2]:
import pandas as pd
import numpy as np
import os


In [3]:
processed_path = r"D:\AlmaBetter\EDAproject\data\proccessed\paisabazaar_processed.csv"
df = pd.read_csv(processed_path)

df.head()


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,5634,3392,1,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,...,Good,809.98,26.822620,265.0,No,49.574949,21.46538,High_spent_Small_value_payments,312.494089,Good
1,5635,3392,2,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,...,Good,809.98,31.944960,266.0,No,49.574949,21.46538,Low_spent_Large_value_payments,284.629162,Good
2,5636,3392,3,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,...,Good,809.98,28.609352,267.0,No,49.574949,21.46538,Low_spent_Medium_value_payments,331.209863,Good
3,5637,3392,4,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,...,Good,809.98,31.377862,268.0,No,49.574949,21.46538,Low_spent_Small_value_payments,223.451310,Good
4,5638,3392,5,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,...,Good,809.98,24.797347,269.0,No,49.574949,21.46538,High_spent_Medium_value_payments,341.489231,Good


### Standardize Column Names

Lowercase

Snake_case

Remove spaces/special characters

In [4]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(r"[^\w]", "", regex=True)
)

df.columns


Index(['id', 'customer_id', 'month', 'name', 'age', 'ssn', 'occupation',
       'annual_income', 'monthly_inhand_salary', 'num_bank_accounts',
       'num_credit_card', 'interest_rate', 'num_of_loan', 'type_of_loan',
       'delay_from_due_date', 'num_of_delayed_payment', 'changed_credit_limit',
       'num_credit_inquiries', 'credit_mix', 'outstanding_debt',
       'credit_utilization_ratio', 'credit_history_age',
       'payment_of_min_amount', 'total_emi_per_month',
       'amount_invested_monthly', 'payment_behaviour', 'monthly_balance',
       'credit_score'],
      dtype='object')

### Correct Data Types

Convert numeric-looking strings and date-like fields if present.

In [5]:
for col in df.columns:
    if df[col].dtype == "object":
        try:
            df[col] = pd.to_numeric(df[col])
        except:
            pass


In [6]:
df.dtypes


id                            int64
customer_id                   int64
month                         int64
name                         object
age                         float64
ssn                         float64
occupation                   object
annual_income               float64
monthly_inhand_salary       float64
num_bank_accounts           float64
num_credit_card             float64
interest_rate               float64
num_of_loan                 float64
type_of_loan                 object
delay_from_due_date         float64
num_of_delayed_payment      float64
changed_credit_limit        float64
num_credit_inquiries        float64
credit_mix                   object
outstanding_debt            float64
credit_utilization_ratio    float64
credit_history_age          float64
payment_of_min_amount        object
total_emi_per_month         float64
amount_invested_monthly     float64
payment_behaviour            object
monthly_balance             float64
credit_score                

### Standardize Categorical Values

Remove inconsistent casing & whitespace

In [7]:
categorical_cols = df.select_dtypes(include="object").columns

for col in categorical_cols:
    df[col] = (
        df[col]
        .str.strip()
        .str.lower()
    )


In [8]:
df[categorical_cols].nunique()


name                     8514
occupation                 15
type_of_loan             4654
credit_mix                  3
payment_of_min_amount       3
payment_behaviour           6
credit_score                3
dtype: int64

### Logical Data Corrections

Ensure no negative values in financial fields

In [9]:
financial_cols = [
    col for col in df.columns
    if any(keyword in col for keyword in ["income", "balance", "amount", "limit", "emi"])
]

# keep only numeric columns to avoid clipping string/object fields
financial_cols = [col for col in financial_cols if pd.api.types.is_numeric_dtype(df[col])]

for col in financial_cols:
    df[col] = df[col].clip(lower=0)


### Feature Engineering

Credit Utilization Ratio


In [10]:
if {"total_credit_limit", "total_credit_used"}.issubset(df.columns):
    df["credit_utilization_ratio"] = (
        df["total_credit_used"] / df["total_credit_limit"]
    ).clip(0, 1)


Debt-to-Income Ratio

In [11]:
if {"monthly_income", "total_emi"}.issubset(df.columns):
    df["debt_to_income_ratio"] = (
        df["total_emi"] / df["monthly_income"]
    ).replace([np.inf, -np.inf], 0)


### Binning Continuous Variables

Income Bands

In [12]:
if "monthly_income" in df.columns:
    df["income_band"] = pd.cut(
        df["monthly_income"],
        bins=[0, 25000, 50000, 100000, np.inf],
        labels=["low", "lower_middle", "upper_middle", "high"]
    )


### Save the Wrangled Dataset

In [13]:
wrangled_path = r"D:\AlmaBetter\EDAproject\data\proccessed\paisabazaar_wrangled.csv"
df.to_csv(wrangled_path, index=False)

print(f"Wrangled dataset saved at:\n{wrangled_path}")


Wrangled dataset saved at:
D:\AlmaBetter\EDAproject\data\proccessed\paisabazaar_wrangled.csv
